# Packing stability problem

In [534]:
from pandas import DataFrame, read_csv
import os
from os import listdir
from os.path import isfile, join
import math
import numpy as np
import random
import copy

In [535]:
file_list = [f for f in listdir(os.getcwd()) if isfile(f) and ('csv' in f)]
file_list

['0-out.csv',
 '1-out.csv',
 '2-out.csv',
 '3-out.csv',
 '4-out.csv',
 '5-out.csv',
 '6-out.csv',
 '7-out.csv',
 '8-out.csv',
 '9-out.csv']

In [576]:
def loadData(file_name):
    data = read_csv(os.getcwd()+'/'+ file_name, sep=',', index_col=False, skiprows = 2, names=['SKU', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2', 'Aisle', 'Weight'])
    return data
file_name = '1-out.csv'
data = loadData(file_name)
data.head(0)

,SKU,X1,Y1,Z1,X2,Y2,Z2,Aisle,Weight
0,790145,0,0,0,600,391,211,6,10727
1,723017,600,0,0,1173,256,185,6,3067
2,747090,0,391,0,500,699,200,6,8267
3,768211,0,391,200,500,694,398,6,8313
4,738985,500,391,0,850,506,261,7,7480
5,704076,600,256,0,1061,371,307,7,11673
6,751518,0,0,211,462,303,326,7,12407
7,704965,500,506,0,832,728,438,6,5200
8,715845,0,0,326,410,314,454,6,11080
9,794864,600,0,185,752,242,626,6,6107


In [577]:
class box():
    def __init__(self, SKU, x_1, y_1, z_1, x_2, y_2, z_2, weight):
        self.SKU = SKU
        self.size = [x_2 - x_1, y_2 - y_1, z_2 - z_1]
        #self.coords = self.size
        #self.rotation = 3
        self.weight = weight
        self.attPoint = [x_1, y_1, z_1]
        self.farCorner = [x_2, y_2, z_2]
    
    #вернуть SKU
    def getSKU(self):
        return self.SKU

    #вернуть длину (x по умолчанию)
    def getLength(self):
        return self.size[0]

    #вернуть ширину (y по умолчанию)
    def getWidth(self):
        return self.size[1]

    #вернуть высоту (z по умолчанию)
    def getHeight(self):
        return self.size[2]
    
    #вернуть периметр основания коробки
    def getPerimeter(self):
        return self.size[0] * 2 + self.size[1] * 2

    #вернуть свои исходные размеры
    def getSize(self):
        return list(self.size)

    #вернуть вес
    def getWeight(self):
        return self.weight

    def __repr__(self):
        return 'Box(SKU=%s, attPoint=%s, farCorner=%s, weight = %s)' % (self.SKU,self.attPoint, self.farCorner, self.weight)

    def getAttPoint(self):
        return list(self.attPoint)

    def getFarCorner(self):
        return list(self.farCorner)

    def getCenterMassX(self):
        return ((self.farCorner[0]+self.attPoint[0]) / 2.0)
    
    def getCenterMassY(self):
        return ((self.farCorner[1]+self.attPoint[1]) / 2.0)
    
    def getCenterMassZ(self):
        return ((self.farCorner[2]+self.attPoint[2]) / 2.0)
        
    def getUpperPlane(self):
        return ((self.attPoint[0],self.attPoint[1], self.farCorner[2]), (self.farCorner[0],self.farCorner[1], self.farCorner[2]))

    def __eq__(self, other):
        if (self.getSKU() == other.getSKU() and self.getFarCorner() == other.getFarCorner() 
                 and self.attPoint() == other.attPoint()):
            return True
        else:
            return False

    def __ne__(self, other):
        if  not (self.farCorner[0] == other.farCorner[0] and 
                 self.farCorner[1] == other.farCorner[1] and 
                 self.farCorner[2] == other.farCorner[2]and 
                 self.attPoint[0] == other.attPoint[0] and 
                 self.attPoint[1] == other.attPoint[1] and 
                 self.attPoint[2] == other.attPoint[2]):
            return True
        else:
            return False

In [578]:
boxes = []

for i in range(0, len(data) - 1):
    #print data['SKU'][i], data['X1'][i], data['Y1'][i], data['Z1'][i], data['X2'][i], data['Y2'][i], data['Z2'][i], data['Weight'][i]
    boxes.append(box(data['SKU'][i], data['X1'][i], data['Y1'][i], data['Z1'][i], data['X2'][i], data['Y2'][i], data['Z2'][i], data['Weight'][i]))
boxes

[Box(SKU=790145, attPoint=[0, 0, 0], farCorner=[600, 391, 211], weight = 10727),
 Box(SKU=723017, attPoint=[600, 0, 0], farCorner=[1173, 256, 185], weight = 3067),
 Box(SKU=747090, attPoint=[0, 391, 0], farCorner=[500, 699, 200], weight = 8267),
 Box(SKU=768211, attPoint=[0, 391, 200], farCorner=[500, 694, 398], weight = 8313),
 Box(SKU=738985, attPoint=[500, 391, 0], farCorner=[850, 506, 261], weight = 7480),
 Box(SKU=704076, attPoint=[600, 256, 0], farCorner=[1061, 371, 307], weight = 11673),
 Box(SKU=751518, attPoint=[0, 0, 211], farCorner=[462, 303, 326], weight = 12407),
 Box(SKU=704965, attPoint=[500, 506, 0], farCorner=[832, 728, 438], weight = 5200),
 Box(SKU=715845, attPoint=[0, 0, 326], farCorner=[410, 314, 454], weight = 11080),
 Box(SKU=794864, attPoint=[600, 0, 185], farCorner=[752, 242, 626], weight = 6107),
 Box(SKU=724311, attPoint=[832, 506, 0], farCorner=[1142, 736, 114], weight = 6047),
 Box(SKU=789728, attPoint=[752, 0, 185], farCorner=[1189, 181, 350], weight = 133

In [579]:
boxess = []
b1 = box(1, 0,0,0,10,10,10, 100)
b2 = box(1, 10,0,0,20,10,10, 100)
boxess.append(b1)
boxess.append(b2)

In [580]:
def getCenterMass(boxes):
    #x_total = boxes[0].getCenterMassX()
    #y_total = boxes[0].getCenterMassY()
    #z_total = boxes[0].getCenterMassZ()
    weight_total = 0
    sum_r_x = 0
    sum_r_y = 0
    sum_r_z = 0
    
    for each in boxes:
        sum_r_x += each.getCenterMassX() * each.getWeight()
        sum_r_y += each.getCenterMassY() * each.getWeight()
        sum_r_z += each.getCenterMassZ() * each.getWeight()
        #x_total = (each.getCenterMassX() * each.getWeight() + x_total * weight_total) / (weight_total + each.getWeight())
        #y_total = (each.getCenterMassY() * each.getWeight() + y_total * weight_total) / (weight_total + each.getWeight())
        #z_total = (each.getCenterMassZ() * each.getWeight() + x_total * weight_total) / (weight_total + each.getWeight())
        weight_total += each.getWeight()
    
    cm_x = sum_r_x / weight_total
    cm_y = sum_r_y / weight_total
    cm_z = sum_r_z / weight_total
    
    return (cm_x, cm_y, cm_z)

In [581]:
getCenterMass(boxes)

(579.6959959893419, 392.04323552276122, 878.38574055916547)

In [582]:
def perimeter_all(boxess):
    perimeter_all = 0
    for each in boxess:
        perimeter_all += each.getPerimeter()
    return perimeter_all

In [583]:
print perimeter_all(boxes)

92654


In [584]:
def perimeter_common(box1, box2):
    
    def getOverlap(a, b):
            return max(0, min(a[1], b[1]) - max(a[0], b[0]))

    def line_x(box1,box2):
        l_x_1 = (box1.attPoint[0], box1.farCorner[0])
        l_x_2 = (box2.attPoint[0], box2.farCorner[0])
        return getOverlap(l_x_1, l_x_2)

    def line_y(box1,box2):
        l_y_1 = (box1.attPoint[1], box1.farCorner[1])
        l_y_2 = (box2.attPoint[1], box2.farCorner[1])
        return getOverlap(l_y_1, l_y_2)
    
    return ((line_x(box1,box2) + line_y(box1,box2)) * 2) if line_x(box1, box2) * line_y(box1, box2) != 0 else 0

In [585]:
def isSameHeight(box1, box2):
    if box1.attPoint[2] == box2.farCorner[2]:
        return True
    else: 
        return False

In [586]:
def perimeter_com_all(boxes):
    per_com_all = 0
    for each in boxes:
        for each1 in boxes:
            if (each != each1 and isSameHeight(each, each1)):
                per_com_all += perimeter_common(each, each1)
    return per_com_all

In [587]:
print perimeter_com_all(boxes)

73304


In [588]:
def stability_coeff(boxes):
    #if getCenterMassHeight(boxes) < 1000 and (perimeter_all(boxes) - perimeter_com_all(boxes)) > 50:
        #return 1
    #else: 
        #return 0
    return getCenterMass(boxes)[2] + (perimeter_all(boxes) - perimeter_com_all(boxes))

In [589]:
print stability_coeff(boxes)

20228.3857406
